# Chemical Equation Balancer

The following code will compute the balanced form of the given chemical equation. The main purpose of this project is to get use to _numpy_ library and understanding the first use case of solving linear equations.

## Import required libraries

In [206]:
import numpy as np

## Get and Parse Inputs
Get and parse inputs to a useable format. 

(In the following documentation, we will be describing the code using the following example:

Input:
```
C H O
C2 H6 + O2 -> C O2 + H2 O
```
)

The following piece of code will parse the given inputs as follows:  
   + _equation_elements_ = `['C', 'H', 'O']`
   + _chemical_equation_ = `C2 H6 + O2 -> C O2 + H2 O`

In [179]:
def get_and_parse_inputs():
    equation_elements = np.array(input().split())
    chemical_equation = input()
    return equation_elements, chemical_equation

## Methods for extracting coefficients
The following methods, will extract coefficients from equation and construct the corresponding augmented matrix. The description of each method is as follows:

+ __extract_elements_coefficients_from_mixture__:
    + given available_elements (`['C', 'H', 'O']` in our example) and a mixture (like `C2 H6` for example), this method will compute and return the coeffient for each element in the given mixture. (will return `[2, 6, 0]` in above example)
       
       
+ __extract_elements_coefficients_from_summation__:
    + given available_elements (`['C', 'H', 'O']` in our example) and a summation (like `C2 H6 + O2` for example), this method will compute the coeffient for each mixture and append it as a new column to the result matrix. (will return `[[2, 0], [6, 0], [0, 2]]` in above example)


In [168]:
def extract_elements_coefficients_from_mixture(available_elements, mixture):
    mixture_elements = np.array(mixture.split())
    element_count = dict.fromkeys(available_elements, 0)
    for element in mixture_elements:
        # given "C5", this will increase "C" element's count by 5
        if len(element) > 1:
            element_count[element[0]] += int(element[1])
        else:
            element_count[element[0]] += 1
    
    
    result_coefficients = np.zeros(available_elements.size)
    for index, element in enumerate(available_elements):
        result_coefficients[index] = element_count[element]
    return result_coefficients

In [118]:
def extract_elements_coefficients_from_summation(available_elements, summation, is_negative=False):
    mixtures = np.array(summation.split('+'))
    
    for index, mixture in enumerate(mixtures):
        mixture_coefficients = extract_elements_coefficients_from_mixture(available_elements, mixture)
        if is_negative:
            mixture_coefficients *= -1
            
        if index == 0:
            result_coefficients = mixture_coefficients
        else:
            # this will append the computed coefficients (mixture_coefficients) 
            # as a new column to the current extracted matrix
            result_coefficients = np.c_[result_coefficients, mixture_coefficients]
            
    return result_coefficients

## Constucting the augmented matrix

+ __construct_augmented_matrix__:
    + Construct the augmented matrix by calling _extract_elements_coefficients_from_summation_ method for reactants and products and appending another column of zeros.
    + In our example:
        + _reactants_ = `C2 H6 + O2 `
        + _products_ = ` C O2 + H2 O`
        + will return `[[2, 0, -1, 0, 0], [6, 0, 0, -2, 0], [0, 2, -2, -1, 0]]`

In [120]:
def construct_augmented_matrix(available_elements, equation):
    reactants, products = equation.split('->')
    reactants_matrix = extract_elements_coefficients_from_summation(available_elements, reactants)
    products_matrix = extract_elements_coefficients_from_summation(available_elements, products, True)
    return np.c_[reactants_matrix, products_matrix, np.zeros(available_elements.size)]

## Compute echelon form

Compute a echelon form of the augmented matrix.

+ __sort_matrix_rows_by_leading_entries__:
    + Sort rows of the given matrix, in such a way that row _i1_ comes higher than _i2_ if the leading entry in _i1_ is further left than the leading entry in _i2_.
    + In our example:
        + _A_ = `[[2, 0, -1, 0, 0], [0, 0, 3, -2, 0], [0, 2, -2, -1, 0]]`
        + will return `[[2, 0, -1, 0, 0], [0, 2, -2, -1, 0]], [0, 0, 3, -2, 0]`
+ __compute_echelon_matrix__:
    + Compute the echelon form of the given matrix in the following way. First sort rows by their leading entries, then choose first unchekced row and scale/subtract rows at below of that in such a way that after applying this operation, all entries below the leading entry in the current row will become zero. Then, mark the current row as checked and continue this algorithm until neither no unchecked row or leading entry's left.
    + In our example:
        + _A_ = `[[2, 0, -1, 0, 0], [0, 2, -2, -1, 0], [6, 0, 0, -2, 0]]`
        + a probable output can be `[[2, 0, -1, 0, 0], [0, 2, -2, -1, 0]], [0, 0, 3, -2, 0]`

In [205]:
def sort_matrix_rows_by_leading_entries(A):
    assert len(A.shape) == 2
    result = A.astype(np.float64)
    
    topmost_row = 0
    for j in range(result.shape[1]):
        for i in range(topmost_row, result.shape[0]):
            if float(A[i, j]) != 0:
                # swap current row with the first untouched row
                result[[topmost_row, i]] = result[[i, topmost_row]]
                topmost_row += 1
                i -= 1
    return result

In [204]:
def compute_echelon_matrix(A):
    assert len(A.shape) == 2
    result = A.astype(np.float64)
    
    topmost_row = 0
    for j in range(result.shape[1]):
        result = sort_matrix_rows_by_leading_entries(result)
        if float(result[topmost_row][j]) == 0:
            # this row and all rows at below are 0 at this column
            continue
            
        # leading entry at this column is set to be at topmost_row-th,
        # so the rows at below must be scaled and then subtracted by current row
        for i in range(topmost_row + 1, result.shape[0]):
            if float(result[i][j]) == 0:
                # this row and all rows at below are 0 at this column
                break
            
            scaling_coefficient = result[topmost_row][j] / result[i][j]
            for col in range(j, result.shape[1]):
                # first scale the element, the subtract it by the candidate entry
                result[i][col] *= scaling_coefficient
                result[i][col] -= result[topmost_row][col]
                
        topmost_row += 1
        if topmost_row == result.shape[0]:
            break
    return result

## Compute reduced echelon form
Transform the computed echelon matrix to the reduced echelon form.

+ __compute_reduced_echelon_form__:
    + Compute the reduced echelon form of the given matrix. First it will construct an echelon form of the matrix using the _compute_echelon_matrix_ method and scale each row such that the leading entry in that row will become 1. Then from bottom to top, it will find an unchecked leading entry and scale/subtract rows at top of it so that all entries in the corresponding column except the one which is leading entry will become zero.
    + In our example:
        + _A_ = `[[2, 0, -1, 0, 0], [0, 0, 3, -2, 0], [0, 2, -2, -1, 0]]`
        + will return `[[1, 0, 0, -0.33333333,  0], [0, 1, 0, -1.16666667, 0], [0, 0, 1, -0.66666667, 0]]`

In [182]:
def compute_reduced_echelon_form(A):
    assert len(A.shape) == 2
    result = compute_echelon_matrix(A)
    
    bottommost_row = result.shape[0] - 1
    for j in range(result.shape[1] - 1, -1, -1):
        if j > 0 and result[bottommost_row][j - 1] != 0:
            continue
        
        # first scale the candidate row such that the leading entry in this row
        # will become 1
        scaling_coefficient = 1.0 / result[bottommost_row][j]
        for col in range(j, result.shape[1]):
            result[bottommost_row][col] *= scaling_coefficient
        
        # then subtract rows at top, such that all entries over the current leading entry
        # become 0
        for i in range(bottommost_row - 1, -1, -1):
            if result[i][j] == 0:
                continue
                
            scaling_coefficient = 1.0 / result[i][j]
            for col in range(0, result.shape[1]):
                result[i][col] = result[i][col] * scaling_coefficient - result[bottommost_row][col]

        bottommost_row -= 1
        if bottommost_row == -1:
            break
    return result

## Extract answers
The final objective is to extract answers after computing reduced echelon matrix.

+ __extract_answers_from_reduced_echelon_matrix__:
    + Compute an arbitrary answer to the given reduced echelon form of matrix. The answer is made by assigning value of 1 to all free variables and calculating others by this hypothesis.
    + In our example:
        + _A_ = `[[1, 0, 0, -0.33333333,  0], [0, 1, 0, -1.16666667, 0], [0, 0, 1, -0.66666667, 0]]`
        + will return `[0,]`

In [207]:
def extract_answers_from_reduced_echelon_matrix(A):
    assert len(A.shape) == 2
    result = np.ones(A.shape[1] - 1)
    
    for i in range(A.shape[0]):
        leading_entry = 0
        while leading_entry < A.shape[1] - 1 and A[i][leading_entry] == 0:
            leading_entry += 1
        if leading_entry == A.shape[1] - 1:
            break
        
        result[leading_entry] = 0
        for j in range(leading_entry + 1, A.shape[1] - 1):
            result[leading_entry] -= A[i][j]
        result[leading_entry] += A[i][A.shape[1] - 1]
    return result

## Wrap up the code
Complete code by calling required predefined methods.

In [213]:
def run_chemical_equation_balancer():
    equation_elements, chemical_equation = get_and_parse_inputs()
    augmented_matrix = construct_augmented_matrix(equation_elements, chemical_equation)
    echelon_matrix = compute_echelon_matrix(augmented_matrix)
    reduced_echelon_matrix = compute_reduced_echelon_form(augmented_matrix)
    answers = extract_answers_from_reduced_echelon_matrix(reduced_echelon_matrix)
    
    print("\n\n- Augmented matrix of the chemical coefficients: \n", augmented_matrix)
    print("\n\n- Echelon form of the augmented matrix: \n", echelon_matrix)
    print("\n\n- Reduced echelon form of the augmented matrix: \n", reduced_echelon_matrix)
    print("\n\n- One arbitrary answer to the chemical equation balancing:")
    for i in range(len(answers)):
        print('-- X' + str(i + 1) + " = " + str(answers[i]))

run_chemical_equation_balancer()

C H O
C2 H6 + O2 -> C O2 + H2 O


- Augmented matrix of the chemical coefficients: 
 [[ 2.  0. -1. -0.  0.]
 [ 6.  0. -0. -2.  0.]
 [ 0.  2. -2. -1.  0.]]


- Echelon form of the augmented matrix: 
 [[ 2.          0.         -1.         -0.          0.        ]
 [ 0.          2.         -2.         -1.          0.        ]
 [ 0.          0.          1.         -0.66666667  0.        ]]


- Reduced echelon form of the augmented matrix: 
 [[ 1.          0.         -0.         -0.33333333  0.        ]
 [-0.          1.         -0.         -1.16666667  0.        ]
 [ 0.          0.          1.         -0.66666667  0.        ]]


- One arbitrary answer to the chemical equation balancing:
-- X1 = 0.3333333333333333
-- X2 = 1.1666666666666665
-- X3 = 0.6666666666666666
-- X4 = 1.0
